In [1]:
import os
import csv
import zipfile
import string
import pandas as pd

## Data Loading

### Load Calls-for-Service Data

In [2]:
# Set location of file
path = os.path.join("..\\Datasets\\Raw_Data\\Calls_for_Service\\")
path

'..\\Datasets\\Raw_Data\\Calls_for_Service\\'

In [3]:
# Get filenames
filenames = os.listdir(path)
filenames

['Calls_for_Service_2012.zip',
 'Calls_for_Service_2013.zip',
 'Calls_for_Service_2014.zip',
 'Calls_for_Service_2015.zip',
 'Calls_for_Service_2016.zip']

In [4]:
# Load data from files in list
dfs = []
for f in filenames:
    zf = zipfile.ZipFile(os.path.join(path,f)) 
    dfs.append(pd.read_csv(zf.open(str.replace(f, 'zip', 'csv')),))

# Merge all df in list
cfs_df = pd.concat(dfs, ignore_index=True)

# Change datatype of column Type_ to String
cfs_df.Type_ = cfs_df.Type_.apply(str)

In [5]:
# Display top 5 rows
cfs_df.head()

,NOPD_Item,Type_,TypeText,Priority,InitialType,InitialTypeText,InitialPriority,MapX,MapY,TimeCreate,...,TimeArrive,TimeClosed,Disposition,DispositionText,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location
0,A0000112,62A,"BURGLAR ALARM, SILEN",2C,NaN,NaN,NaN,3683627,532625,1/1/2012 0:00,...,NaN,1/1/2012 0:33,NAT,NECESSARY ACTION TAKEN,NaN,NaN,009XX Decatur St,70116.0,8,"(29.958469303316875, -90.0613152964016)"
1,A0000412,94,DISCHARGING FIREARMS,2B,NaN,NaN,NaN,3732996,562418,1/1/2012 0:00,...,1/1/2012 0:16,1/1/2012 0:30,UNF,UNFOUNDED,NaN,NaN,147XX Chef Menteur Hwy,70129.0,7,"(30.038788769111676, -89.90425047516077)"
2,A0000212,103,DISTURBANCE (OTHER),1C,NaN,NaN,NaN,3687688,548824,1/1/2012 0:01,...,1/1/2012 0:01,1/1/2012 0:19,NAT,NECESSARY ACTION TAKEN,NaN,NaN,038XX Gentilly Blvd,70122.0,3,"(30.002886229898206, -90.04791794333323)"
3,A0000712,21,COMPLAINT OTHER,1H,NaN,NaN,NaN,3670776,521242,1/1/2012 0:01,...,NaN,1/1/2012 0:20,NAT,NECESSARY ACTION TAKEN,NaN,NaN,Carondelet St & Napoleon Ave,70115.0,2,"(29.927555772946167, -90.10228161624175)"
4,A0000512,62A,"BURGLAR ALARM, SILEN",2C,NaN,NaN,NaN,3665739,549621,1/1/2012 0:01,...,1/1/2012 0:09,1/1/2012 1:55,NAT,NECESSARY ACTION TAKEN,NaN,NaN,002XX W Harrison Ave,70124.0,3,"(30.005736477457617, -90.11723146931276)"


In [6]:
# Number of records
len(cfs_df)

2252907

In [7]:
# Column Names
cfs_df.columns

Index([u'NOPD_Item', u'Type_', u'TypeText', u'Priority', u'InitialType',
       u'InitialTypeText', u'InitialPriority', u'MapX', u'MapY', u'TimeCreate',
       u'TimeDispatch', u'TimeArrive', u'TimeClosed', u'Disposition',
       u'DispositionText', u'SelfInitiated', u'Beat', u'BLOCK_ADDRESS', u'Zip',
       u'PoliceDistrict', u'Location'],
      dtype='object')

In [8]:
cfs_beat = cfs_df[['NOPD_Item','Beat']]

In [9]:
cfs_beat.head()

,NOPD_Item,Beat
0,A0000112,NaN
1,A0000412,NaN
2,A0000212,NaN
3,A0000712,NaN
4,A0000512,NaN


In [10]:
# Filter records where beat values are not null
cfs_beat = cfs_beat[cfs_beat.Beat.notnull()]

In [11]:
len(cfs_beat.NOPD_Item)

1283208

In [12]:
# Unique NOPD Item
len(set(cfs_beat.NOPD_Item))

1283194

# Merging with Stop and Search

In [13]:
sns_file_path = "..\\Datasets\\Raw_Data\\Stop_and_Search\\Stop_and_Search__Field_Interviews_.csv"

In [14]:
# Load the SNS Data
sns_df = pd.read_csv(sns_file_path) 

C:\Users\Nikhil\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (1,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
sns_df.head()

,FieldInterviewID,NOPD_Item,EventDate,District,Zone,OfficerAssignment,StopDescription,ActionsTaken,VehicleYear,VehicleMake,...,SubjectWeight,SubjectEyeColor,SubjectHairColor,SubjectDriverLicState,CreatedDateTime,LastModifiedDateTime,Longitude,Latitude,Zip,BlockAddress
0,17415,NaN,01/01/2010 01:11:00 AM,6,E,6th District,TRAFFIC VIOLATION,NaN,2005.0,DODGE,...,160.0,Brown,Black,LA,01/01/2010 01:26:26 AM,NaN,0.0,0.0,NaN,NaN
1,17416,NaN,01/01/2010 02:06:00 AM,5,D,5th District,CALL FOR SERVICE,NaN,NaN,NaN,...,140.0,Brown,Black,NaN,01/01/2010 02:27:38 AM,NaN,0.0,0.0,NaN,NaN
2,17416,NaN,01/01/2010 02:06:00 AM,5,D,5th District,CALL FOR SERVICE,NaN,NaN,NaN,...,145.0,Brown,Black,NaN,01/01/2010 02:27:38 AM,NaN,0.0,0.0,NaN,NaN
3,17416,NaN,01/01/2010 02:06:00 AM,5,D,5th District,CALL FOR SERVICE,NaN,NaN,NaN,...,140.0,Brown,Black,NaN,01/01/2010 02:27:38 AM,NaN,0.0,0.0,NaN,NaN
4,17416,NaN,01/01/2010 02:06:00 AM,5,D,5th District,CALL FOR SERVICE,NaN,NaN,NaN,...,140.0,Brown,Black,NaN,01/01/2010 02:27:38 AM,NaN,0.0,0.0,NaN,NaN


In [16]:
sns_df.EventDate = pd.to_datetime(sns_df.EventDate)

In [17]:
sns_df.columns

Index([u'FieldInterviewID', u'NOPD_Item', u'EventDate', u'District', u'Zone',
       u'OfficerAssignment', u'StopDescription', u'ActionsTaken',
       u'VehicleYear', u'VehicleMake', u'VehicleModel', u'VehicleStyle',
       u'VehicleColor', u'SubjectID', u'SubjectRace', u'SubjectGender',
       u'SubjectAge', u'SubjectHasPhotoID', u'SubjectHeight', u'SubjectWeight',
       u'SubjectEyeColor', u'SubjectHairColor', u'SubjectDriverLicState',
       u'CreatedDateTime', u'LastModifiedDateTime', u'Longitude', u'Latitude',
       u'Zip', u'BlockAddress'],
      dtype='object')

In [18]:
sns_df = sns_df[['NOPD_Item', 'EventDate', 'Zip']]

In [19]:
sns_df.head()

,NOPD_Item,EventDate,Zip
0,NaN,2010-01-01 01:11:00,NaN
1,NaN,2010-01-01 02:06:00,NaN
2,NaN,2010-01-01 02:06:00,NaN
3,NaN,2010-01-01 02:06:00,NaN
4,NaN,2010-01-01 02:06:00,NaN


In [20]:
len(sns_df.NOPD_Item)

430607

In [21]:
sns_df['EventYear'] = [val.year for val in sns_df.EventDate]

Select date between 2012 - 2016

In [22]:
import datetime
sns_df = sns_df[(sns_df.EventDate >= datetime.date(2012,1,1)) & (sns_df.EventDate < datetime.date(2017,1,1))]

In [23]:
max(sns_df.EventDate)

Timestamp('2016-12-31 23:25:00')

In [24]:
min(sns_df.EventDate)

Timestamp('2012-01-01 00:05:00')

In [25]:
# Unique NOPD Item
tot_sns = len(set(sns_df.NOPD_Item))
tot_sns

215854

In [26]:
# Merge SNS and CFS Data
cfs_sns_df = pd.merge(sns_df, cfs_beat , how='left', on='NOPD_Item')

In [27]:
cfs_sns_df.head()

,NOPD_Item,EventDate,Zip,EventYear,Beat
0,F0010411,2013-05-31 01:49:00,70112.0,2013,NaN
1,F0010411,2013-05-31 01:49:00,70112.0,2013,NaN
2,B0000012,2012-02-17 01:00:00,NaN,2012,NaN
3,B0000012,2012-02-17 01:00:00,NaN,2012,NaN
4,E0187912,2012-05-02 08:11:00,70115.0,2012,NaN


In [28]:
# Unique NOPD Item
tot_sns_beat = len(set(cfs_sns_df.NOPD_Item[cfs_sns_df.Beat.notnull()]))
tot_sns_beat

131383

In [29]:
sns_beat_available = tot_sns_beat*100.0/tot_sns

In [30]:
100 - sns_beat_available

39.13339572118191